In [20]:
import os
import pickle

## Prepare data 

#### 1.Load data into list 

In [21]:
data_path = '../../data/10_translation/srt'
data_cn_path = os.path.join(data_path,'cn')
data_en_path = os.path.join(data_path,'en')

In [22]:
en_files_names = os.listdir(data_en_path)
cn_files_names = os.listdir(data_en_path)
en_files_dict = {int(f.split('_')[0]):f for f in en_files_names}
cn_files_dict = {int(f.split('_')[0]):f for f in cn_files_names}

In [23]:
en_files_dict[0]

"0_'71.2014.BluRay.720p.x264.AC3.╖▒╠σ&╙ó╬─.txt"

In [24]:
## define a function to read certain number of fines and make them into a list
def read_files(file_path,file_dict,num_files):
    sentence_list = []
    for i in range(num_files):
        path = os.path.join(file_path,file_dict[i])
        with open(path, 'r') as f:
            lines = f.readlines()
            #print(len(lines))
            sentence_list.extend(lines)
            
    return sentence_list

In [25]:
## number of files we want to read 
num_files = 1000
train_X = read_files(data_en_path,en_files_dict,num_files)
train_Y = read_files(data_cn_path,cn_files_dict,num_files)

In [26]:
## check if lines are alined 
print((len(train_X),len(train_Y)))

(417205, 417205)


In [27]:
x = 1
print(train_X[x])
print(train_Y[x])

That's it! Into him, Hook!

就是这样  靠近点  胡克



#### 2 Clean up data  - create vocab

In [28]:
import re
from collections import Counter

In [29]:
_WORD_SPLIT = re.compile(r"([.,!?\"':;])")                  ## make all punctuation into its' own token
_MIX_SPLIT = re.compile(r"[\w\d]|-+|[\u4e00-\ufaff]|[^\s]") ## keep all chinese and english words 
_DIGIT_RE = re.compile(r"\d+")

def en_tokenize_setence(se):
    words = []
    for s in se.split():
        words.extend(_WORD_SPLIT.split(s))
    return words

def cn_tokenize_setence(se):
    return _MIX_SPLIT.findall(se)

In [30]:
X_sents = [en_tokenize_setence(x) for x in train_X]
X_voc_counter = Counter([word for sentance in X_sents for word in sentance])
X_voc = sorted(X_voc_counter, key=X_voc_counter.get, reverse=True)

Y_sents = [cn_tokenize_setence(y) for y in train_Y]
Y_voc_counter = Counter([word for sentance in Y_sents for word in sentance])
Y_voc = sorted(Y_voc_counter, key=Y_voc_counter.get, reverse=True)

In [31]:
special_words = ['<pad>', '<unk>', '<go>',  '<eos>']
def get_voc_int_map(voc,special_words):
    int_to_vocab = {word_i: word for word_i, word in enumerate(special_words + voc)}
    vocab_to_int = {word: word_i for word_i, word in int_to_vocab.items()}
    return int_to_vocab,vocab_to_int


In [32]:
### get int voc map
en_int_to_vocab,en_vocab_to_int = get_voc_int_map(X_voc,special_words)
cn_int_to_vocab,cn_vocab_to_int = get_voc_int_map(Y_voc,special_words)

In [33]:
## function to convert words to ids 
def sentence_to_token_ids(sentences, vocabulary):
    letter_ids = [[vocabulary.get(letter,vocabulary['<unk>']) for letter in line] for line in sentences]
    return letter_ids

In [34]:
X = sentence_to_token_ids(X_sents, en_vocab_to_int)
Y = sentence_to_token_ids(Y_sents, cn_vocab_to_int)

In [35]:
X_max_length = max([len(line) for line in X])
Y_max_length = max([len(line) for line in Y])

In [36]:
def save_dataset(file_path,obj):
    with open(file_path,'wb') as f:
        pickle.dump(obj,f)

In [37]:
obj = (X,Y,X_max_length,Y_max_length,en_int_to_vocab,en_vocab_to_int,cn_int_to_vocab,cn_vocab_to_int)
save_dataset('cn2en.p',obj)

In [38]:
def read_dataset(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)
    
#X,Y,X_max_length,Y_max_length,en_int_to_vocab,en_vocab_to_int,cn_int_to_vocab,cn_vocab_to_int = read_dataset('cn2en.p')